# The Art of Dataset Design
## How to Build Tables That Support Any Analysis
#### By William James, Staff Data Scientist, ex-Kry, ex-Spotify

# Agenda
* Super engaging hook story about Spotify launching in the US
* Less engaging agenda walk through
* Short intro to data warehouse best practices
* Introducing `signals` - my favourite analytics table design
* How to create your very own `signals` dataset (warning technical!)

Notebook available at https://github.com/oldpa/presentations

In [1]:
from IPython.display import Image
from IPython.core.display import HTML

## Datawarehouse: A centralized repository for storing and managing large volumes of data from different sources

### Facilitates decision-making and reporting by providing a single source of truth for business data

In [2]:
Image(url='https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_15-DataWarehouse-WHITEBG.png', width=400)

## To have order in your datawarehouse, separate tables into facts and dims

### Facts: Quantitative, numerical data representing business metrics (e.g., sales, revenue, profit)

### Dimensions: Descriptive, categorical data providing context to facts (e.g., product, customer, location)

### Star schema: Consists of a central fact table connected to multiple dimension tables

In [3]:
Image(url="https://learn.microsoft.com/en-us/power-bi/guidance/media/star-schema/star-schema-example2.png",width=300)

## Reading suggestion: Ralph Kimballs The Data Warehouse Tool kit

In [4]:
Image(url="https://m.media-amazon.com/images/I/611LSmsnR2S._AC_UF1000,1000_QL80_.jpg", width=400)

## Example: TODO app - finding the "Aha"-moment

```SQL
-- Is there a correlation between first week usage and revenue?
SELECT
    create_todo_first_7_days,
    sum(revenue_last_28_days)
FROM
    signals
WHERE days_since_signup > 28
GROUP BY 1
ORDER BY 1
```

# Imports

In [5]:
import sqlite3
import random
import time
import pandas as pd
from datetime import datetime, timedelta

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline

## Step 0: Create mock event data

In [6]:
# Create a connection and a cursor
!rm events.db
conn = sqlite3.connect('events.db')
cursor = conn.cursor()

# Create the `facts.events` table

cursor.execute('''drop table if exists events''')
cursor.execute('''CREATE TABLE IF NOT EXISTS events
                 (timestamp TEXT, user_id INTEGER, event_name TEXT, value REAL)''')

# Generate events for 100 users
event_types = ["dau", "login", "create_todo", "complete_todo", "move_todo", "delete_todo", "view_ad", "revenue"]
num_users = 100
num_days = 180

# Helper function to generate random date within the last 180 days
def random_date():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=num_days)
    random_seconds = random.randint(0, int((end_date - start_date).total_seconds()))
    return start_date + timedelta(seconds=random_seconds)

def random_time(date):
    return date + timedelta(seconds = random.randint(0, 3600 * 24))

for user_id in range(1, num_users + 1):
    # Simulate users having different engagement
    engagement = random.random()
    signup_date = random_date()
    cursor.execute("INSERT INTO events (timestamp, user_id, event_name, value) VALUES (?, ?, ?, ?)", (signup_date, user_id, 'login', None))
    for i in range(num_days):
        date = signup_date + timedelta(i)
        if random.random() > engagement:
            for _ in range(1 + int(engagement * 10)):
                timestamp = random_time(date).strftime("%Y-%m-%d %H:%M:%S")
                event_name = random.choice(event_types)

                if event_name == "revenue":
                    value = round(random.uniform(0.5, 50), 2)
                else:
                    value = None

                cursor.execute("INSERT INTO events (timestamp, user_id, event_name, value) VALUES (?, ?, ?, ?)", (timestamp, user_id, event_name, value))

# Commit the changes and close the connection
conn.commit()



## The starting point is event data, one row per event on user_id level

In [7]:
pd.read_sql_query("SELECT * FROM events limit 10", conn)

,timestamp,user_id,event_name,value
0,2022-10-24 01:01:30.630475,1,login,NaN
1,2022-11-21 14:06:06,1,complete_todo,NaN
2,2022-11-21 22:03:51,1,revenue,29.07
3,2022-11-21 16:14:37,1,delete_todo,NaN
4,2022-11-21 09:08:48,1,complete_todo,NaN
5,2022-11-21 15:28:56,1,create_todo,NaN
6,2022-11-21 23:09:15,1,move_todo,NaN
7,2022-11-21 09:06:35,1,delete_todo,NaN
8,2022-11-21 11:54:28,1,dau,NaN
9,2022-11-21 04:01:39,1,move_todo,NaN


## Step 1: Create aggregated events per day and user

In [8]:
conn.execute('drop table if exists daily_events')
conn.execute("""
create table daily_events as
SELECT
user_id,
date(timestamp) as date,
count(case when event_name ='login' then 1 end) as login,
count(case when event_name ='create_todo' then 1 end) as create_todo,
count(case when event_name ='complete_todo' then 1 end) as complete_todo,
count(case when event_name ='move_todo' then 1 end) as move_todo,
count(case when event_name ='delete_todo' then 1 end) as delete_todo,
count(case when event_name ='view_ad' then 1 end) as view_ad,
sum(case when event_name ='revenue' then value end) as revenue,
cast(
    count(
        case
            when event_name in ('create_todo', 'complete_todo') then 1
        end)
> 0 as integer) as dau
from events
group by 1,2
""");

In [9]:
pd.read_sql_query("select * from daily_events limit 10", conn)

,user_id,date,login,create_todo,complete_todo,move_todo,delete_todo,view_ad,revenue,dau
0,1,2022-10-24,1,0,0,0,0,0,NaN,0
1,1,2022-11-21,0,1,2,2,2,0,29.07,1
2,1,2022-11-25,1,1,0,2,0,2,24.11,1
3,1,2022-11-29,1,1,0,0,2,2,43.54,1
4,1,2022-12-02,1,2,1,1,0,1,12.83,1
5,1,2022-12-03,1,0,1,0,0,0,NaN,1
6,1,2022-12-29,1,2,1,1,0,1,36.81,1
7,1,2022-12-31,2,3,0,2,2,0,NaN,1
8,1,2023-01-09,0,4,0,0,1,0,22.27,1
9,1,2023-01-10,0,0,1,0,0,0,NaN,1


## Step 2: Create signals dataset

## First create a cross join between all dates and all users

In [10]:
SIGNALS_QUERY = """
create table signals as
WITH RECURSIVE
    -- Create a list of consecutive dates
    dates (date) AS (
        SELECT date(julianday('now') - 179)
        UNION ALL
        SELECT date(julianday(date) + 1)
        FROM dates
        WHERE date < date('now')
    ),
    -- Create a list of unique users
    users AS (
        SELECT user_id, min(date) as signup_date
        FROM daily_events group by 1
    -- Cross join all users and dates, one row per user_id and date
    ), user_dates as (
        SELECT users.user_id, dates.date, users.signup_date
        FROM users
        CROSS JOIN dates
        where dates.date >= users.signup_date
    )
"""

## Left join this data with the daily events

In [11]:
SIGNALS_QUERY += """
    SELECT
    ud.user_id,
    ud.date,
    round(
        julianday(ud.date) - julianday(ud.signup_date)
    ) as days_since_signup,
    
    --<events go here>--
    
    current_timestamp as etl_at
    FROM user_dates ud
    left join daily_events e on e.user_id = ud.user_id and e.date = ud.date
"""

## Add login related events

In [12]:
sum_and_partition = "sum(e.login) over (partition by ud.user_id order by ud.date rows between"
EVENTS_QUERY = f"""
--login events
{sum_and_partition} current row and current row) as login_today,
{sum_and_partition} 1 preceding and 1 preceding) as login_yesterday,
{sum_and_partition} 6 preceding and current row) as login_last_7_days,
{sum_and_partition} 27 preceding and current row) as login_last_28_days,
{sum_and_partition} unbounded preceding and 0 following) as login_first_day,
{sum_and_partition} unbounded preceding and 6 following) as login_first_7_days,
"""

## Add for all events

In [13]:
ALL_EVENTS_QUERY = ""
event_types = ["dau", "login", "create_todo", "complete_todo", "move_todo", "delete_todo", "view_ad", "revenue"]
for event in event_types:
    ALL_EVENTS_QUERY += EVENTS_QUERY.replace('login', event)
SIGNALS_QUERY = SIGNALS_QUERY.replace('--<events go here>--', ALL_EVENTS_QUERY)
conn.execute('drop table if exists signals');
conn.execute(SIGNALS_QUERY);

## Thats it! This is what the table looks like:

In [14]:
pd.read_sql_query("""select * from signals limit 10""", conn).T

,0,1,2,3,4,5,6,7,8,9
user_id,1,1,1,1,1,1,1,1,1,1
date,2022-10-24,2022-10-25,2022-10-26,2022-10-27,2022-10-28,2022-10-29,2022-10-30,2022-10-31,2022-11-01,2022-11-02
days_since_signup,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
dau_today,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dau_yesterday,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dau_last_7_days,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
dau_last_28_days,0,0,0,0,0,0,0,0,0,0
dau_first_day,0,0,0,0,0,0,0,0,0,0
dau_first_7_days,0,0,0,0,0,0,0,0,0,0
login_today,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Example: Easily calculate the basics: DAU, MAU

In [15]:
pd.read_sql_query("""
select
    date,
    count(case when dau_today > 0 then 1 end) as dau,
    count(case when dau_last_28_days > 0 then 1 end) as mau
from 
    signals
group by 1
order by 1 desc
""", conn).head(3)

,date,dau,mau
0,2023-03-29,29,96
1,2023-03-28,26,95
2,2023-03-27,34,95


## Example: Filter out users based on a complex combination of signals

In [16]:
# Get users that started but didnt finish the new user journey
pd.read_sql_query("""
select
    count(1)
from 
    signals
    where
    days_since_signup = 7 and
    create_todo_last_7_days > 0
    and complete_todo_last_7_days = 0
""", conn)

,count(1)
0,10


## Example: Revenue (LTV) 28 days after signup

In [17]:
pd.read_sql_query("""
select
    date,
    sum(revenue_last_28_days) as ltv_d28
from 
    signals
    where
    days_since_signup = 27
group by 1 order by 1 desc
""", conn).head(3)

,date,ltv_d28
0,2023-03-26,15.18
1,2023-03-25,372.53
2,2023-03-24,289.04


## Thank you for listening!